# Langchain Agents

Using agents we are able to give LLMs access to tools. These tools allow us to search the web, do maths, run code, and much more. Tools are utility chains under the hood.


To define a agent:
- define the tools to choose from
- define the llm for reasoning
- define the agent type (the prompt)


Available agent types:

1. Zero Shot React: does not have memory
2. Conversational React Description: can remember things we have talked about
3. React Docstore: Search and look up on a doc (wikipedia)
4. Self Ask With Search: Extra information with a search engine. Ask follow-up questions and use the search to get intermediate answers.

In [ ]:
# ! pip install langchain, google-search_results

In [1]:
from langchain.chat_models import ChatOpenAI
from getpass import getpass

OPENAI_API_KEY = getpass()

llm = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    model="gpt-4",
    temperature=0
)

In [2]:
from langchain.callbacks import get_openai_callback

def chat_and_count(agent, query):
    with get_openai_callback() as cb:
        result = agent(query)
        print(f"Spent a total of {cb.total_tokens} tokens")

    return result

In [4]:
from langchain import SerpAPIWrapper 
# Use google search to get the results, need a serpapi key.

SERPAPI_KEY = getpass("Enter your SerpAPI key: ")
search = SerpAPIWrapper(serpapi_api_key=SERPAPI_KEY)

In [14]:
from langchain.agents import initialize_agent, Tool

tools = [
    Tool(
        name="Intermediate Answer",
        func=search.run,
        description="Searches for an answer to your question on the internet"
    )
]

### self-ask-with-search agent

In [15]:
self_ask_with_search_agent = initialize_agent(
    tools, 
    llm, 
    agent="self-ask-with-search", 
    verbose=True)

In [16]:
chat_and_count(self_ask_with_search_agent, "Now is July 11th, 2023, Who won the 2023 xgame men's superpipe final?")



> Entering new  chain...
Yes.
Follow up: When was the 2023 xgame men's superpipe final?
Intermediate answer: Winter X Games Aspen 2023: All results and medals - complete list. Here's your complete guide to all ... Monster Energy Men's Snowboard SuperPipe Podium!
Follow up: Who won the Monster Energy Men's Snowboard SuperPipe at the Winter X Games Aspen 2023?
Intermediate answer: Men's snowboard superpipe. 1. Gold - Scotty James 2. Silver - Jan Scherrer 3. Bronze - Valentino Guseli 4. Hirano Ruka 5.
So the final answer is: Scotty James

> Finished chain.
Spent a total of 1420 tokens


{'input': "Now is July 11th, 2023, Who won the 2023 xgame men's superpipe final?",
 'output': 'Scotty James'}

In [17]:
chat_and_count(self_ask_with_search_agent, "Now is July 11th, 2023, When is the deadline for filing 2023 federal tax for california residents impacted by the winter storm?")



> Entering new  chain...
Yes.
Follow up: What is the usual deadline for filing federal tax?
Intermediate answer: The tax deadline typically falls on April 15 each year, but can be delayed if it falls on a weekend or holiday. Missing the tax deadline can have consequences like penalties and interest. April 18, 2023 - Deadline to File Form 4868 and request an extension.
Follow up: Was there an extension announced for California residents impacted by the winter storm?
Intermediate answer: The IRS announced tax relief for Californians affected by these winter storms. Taxpayers affected by these storms qualify for an extension to October 16, 2023, to file individual and business tax returns and make certain tax payments. This includes: Individuals whose tax returns and payments are due on April 18, 2023.
So the final answer is: October 16, 2023

> Finished chain.
Spent a total of 1485 tokens


{'input': 'Now is July 11th, 2023, When is the deadline for filing 2023 federal tax for california residents impacted by the winter storm?',
 'output': 'October 16, 2023'}

### zero-shot-rect-description agent
Testing out llm's capability in selecting one of the two tools

self_ask_with_search_agent allows only one tool,

Use zero-shot-rect-description agent instead

In [18]:
from langchain.agents import load_tools

math_tools = load_tools(
    ["llm-math"],
    llm=llm,
)

In [20]:
tools.extend(math_tools)

In [23]:
zero_shot_react_agent = initialize_agent(
    tools, 
    llm, 
    agent="zero-shot-react-description", 
    verbose=True)

In [24]:
chat_and_count(zero_shot_react_agent, "what is the result of 345 * 789?")

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')).




> Entering new  chain...
This is a multiplication problem. I can use the calculator to solve it.
Action: Calculator
Action Input: 345 * 789
Observation: Answer: 272205
Thought:I now know the final answer
Final Answer: 272205

> Finished chain.
Spent a total of 636 tokens


{'input': 'what is the result of 345 * 789?', 'output': '272205'}

In [26]:
chat_and_count(zero_shot_react_agent, "what is the product of the ages of elon musk and mark zuckberg?")

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')).




> Entering new  chain...
First, I need to find out the current ages of Elon Musk and Mark Zuckerberg.
Action: Intermediate Answer
Action Input: What is the current age of Elon Musk?
Observation: 52 years
Thought:Now I know Elon Musk's age, I need to find out Mark Zuckerberg's age.
Action: Intermediate Answer
Action Input: What is the current age of Mark Zuckerberg?
Observation: 39 years
Thought:Now that I know the ages of both Elon Musk and Mark Zuckerberg, I can calculate the product of their ages.
Action: Calculator
Action Input: 52 * 39
Observation: Answer: 2028
Thought:I now know the final answer
Final Answer: The product of the ages of Elon Musk and Mark Zuckerberg is 2028.

> Finished chain.
Spent a total of 1311 tokens


{'input': 'what is the product of the ages of elon musk and mark zuckberg?',
 'output': 'The product of the ages of Elon Musk and Mark Zuckerberg is 2028.'}

In [30]:
print(zero_shot_react_agent.agent.llm_chain.prompt.template)

Answer the following questions as best you can. You have access to the following tools:

Intermediate Answer: Searches for an answer to your question on the internet
Calculator: Useful for when you need to answer questions about math.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Intermediate Answer, Calculator]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}
